In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import seaborn as sns
import pylab

import warnings
warnings.filterwarnings(action='ignore')

#### 개인타자

In [2]:
# 개인타자 data
P_H_16 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_H_2016.csv',encoding='CP949')
P_H_17 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_H_2017.csv',encoding='CP949')
P_H_18 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_H_2018.csv',encoding='CP949')
P_H_19 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_H_2019.csv',encoding='CP949')
P_H_20 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_H_2020.csv',encoding='CP949')

In [3]:
PH = pd.concat([P_H_16, P_H_17, P_H_18, P_H_19, P_H_20])

In [4]:
PH.reset_index(drop=True, inplace=True)

#### 개인투수

In [5]:
# 개인투수 data
P_P_16 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_P_2016.csv',encoding='CP949')
P_P_17 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_P_2017.csv',encoding='CP949')
P_P_18 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_P_2018.csv',encoding='CP949')
P_P_19 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_P_2019.csv',encoding='CP949')
P_P_20 = pd.read_csv('C:/Users/dudco/Desktop/2020bigcontest/2020_bigcontest_data/P_P_2020.csv',encoding='CP949')

In [6]:
PP = pd.concat([P_P_16, P_P_17, P_P_18, P_P_19, P_P_20])

In [7]:
PP.reset_index(drop=True, inplace=True)

### 개인투수 파생변수 만들기

#### INN 이닝수

In [8]:
PP['INN'] = PP['INN2'] / 3

#### ERA 평균자책점

In [9]:
PP['ERA_n'] = PP['ER']*9
PP['ERA_d'] = PP['INN']

#### FIP 

In [10]:
PP['FIP_n'] = (PP['HR']*13 + (PP['BB']+PP['HP']-PP['IB'])*3 - PP['KK']*2) + 3.2*PP['INN']
PP['FIP_d'] = PP['INN']

#### KFIP

In [11]:
PP['KFIP_n'] = (PP['HR']*14 + (PP['BB']+PP['HP'])*3 - PP['KK']) + 3.2*PP['INN'] 
PP['KFIP_d'] = PP['INN']

# 음수를 없애기 위해 가중치 1 사용

#### WHIP

In [12]:
PP['WHIP_n'] = PP['HIT'] + PP['BB']
PP['WHIP_d'] = PP['INN']

#### PTB_ERC용

In [13]:
PP['PTB'] = 0.89*(1.255*(PP['HIT']-PP['HR'])+4*PP['HR']) + 0.56*(PP['BB']+PP['HP'])

#### ERC

In [14]:
PP['ERC_n'] = (PP['HIT']+PP['BB']+PP['HP'])*PP['PTB']*9
PP['ERC_d'] = PP['PA']*PP['INN']

# -0.56 하거나 *0.75 해야함

#### LOB_RT 

In [15]:
PP['LOB_n'] = (PP['HIT'] + PP['BB'] + PP['HP'] - PP['R'])
PP['LOB_d'] = (PP['HIT'] + PP['BB'] + PP['HP'] - 1.4*PP['HR'])

#### KwERA

In [16]:
# 5.4 - (PP['KwERA_n']/PP['KwERA_d'])

PP['KwERA_n'] = 12*(PP['KK'] - PP['BB'])
PP['KwERA_d'] = PP['PA']

#### BABIP

In [17]:
PP['P_BABIP_n'] = PP['HIT'] - PP['HR']
PP['P_BABIP_d'] = PP['AB'] - PP['KK']- PP['HR'] + PP['SF']

#### 사이영포인트

In [18]:
PP['SAP'] = (0.55*PP['INN']-PP['ER'])+PP['KK']/10

#### K/BB

In [19]:
PP['K/BB']=PP['BB']/PP['KK']

#### H1 1루타

In [20]:
PP['P_H1'] = PP['HIT'] - PP['H2'] - PP['H3'] - PP['HR']

#### 이닝당 HIT 가중치

In [21]:
PP['YANG_n'] = PP['P_H1'] + PP['H2']*2 + PP['H3']*3 + PP['HR']*4
PP['YANG_d'] = PP['INN']

### 개인투수 7묶음으로 나누기

In [94]:
q=[
    (PP['G_ID']>='20160401HHLG0') & (PP['G_ID']<='20160501NCLT0'),
    (PP['G_ID']>='20160501OBHT0') & (PP['G_ID']<='20160526SKNC0'),
    (PP['G_ID']>='20160527LGOB0') & (PP['G_ID']<='20160619SKLT0'),
    (PP['G_ID']>='20160619WOHH0') & (PP['G_ID']<='20160720SSOB0'),
    (PP['G_ID']>='20160721HTLT0') & (PP['G_ID']<='20160814HHHT0'),
    (PP['G_ID']>='20160814KTNC0') & (PP['G_ID']<='20160908WOSK0'),
    (PP['G_ID']>='20160909KTHH0') & (PP['G_ID']<='20161009WOLT0'),
    (PP['G_ID']>='20170331HHOB0') & (PP['G_ID']<='20170428WOHH0'),
    (PP['G_ID']>='20170429LGKT0') & (PP['G_ID']<='20170524HTHH0'),
    (PP['G_ID']>='20170524KTSS0') & (PP['G_ID']<='20170616LTWO0'),
    (PP['G_ID']>='20170616NCOB0') & (PP['G_ID']<='20170712WOOB0'),
    (PP['G_ID']>='20170713LGSK0') & (PP['G_ID']<='20170809NCSK0'),
    (PP['G_ID']>='20170809WOHT0') & (PP['G_ID']<='20170903LGNC0'),
    (PP['G_ID']>='20170903SKKT0') & (PP['G_ID']<='20171003WOSS0'),
    (PP['G_ID']>='20180324HHWO0') & (PP['G_ID']<='20180422LGNC0'),
    (PP['G_ID']>='20180422SKLT0') & (PP['G_ID']<='20180518SSWO0'),
    (PP['G_ID']>='20180519HHLG0') & (PP['G_ID']<='20180612KTOB0'),
    (PP['G_ID']>='20180612LGNC0') & (PP['G_ID']<='20180707NCWO0'),
    (PP['G_ID']>='20180707SSOB0') & (PP['G_ID']<='20180804SSLT0'),
    (PP['G_ID']>='20180804WOKT0') & (PP['G_ID']<='20180915SKHT0'),
    (PP['G_ID']>='20180915SSKT0') & (PP['G_ID']<='20181014OBLT0'),
    (PP['G_ID']>='20190323HHOB0') & (PP['G_ID']<='20190420WOLG0'),
    (PP['G_ID']>='20190421KTLT0') & (PP['G_ID']<='20190516LGLT0'),
    (PP['G_ID']>='20190516SKNC0') & (PP['G_ID']<='20190608WOOB0'),
    (PP['G_ID']>='20190609HTNC0') & (PP['G_ID']<='20190704LTSK0'),
    (PP['G_ID']>='20190704NCHT0') & (PP['G_ID']<='20190807LGHT0'),
    (PP['G_ID']>='20190807SSNC0') & (PP['G_ID']<='20190831SSOB0'),
    (PP['G_ID']>='20190831WONC0') & (PP['G_ID']<='20191001WOLT0'),
    (PP['G_ID']>='20200505HHSK0') & (PP['G_ID']<='20200602SKNC0'),
    (PP['G_ID']>='20200602SSLG0') & (PP['G_ID']<='20200626HTWO0'),
    (PP['G_ID']>='20200626KTHH0') & (PP['G_ID']<='20200719WOSK0')

]
choicelist =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
PP['q']=np.select(q, choicelist)

In [95]:
# 분모, 분자 상관유무에 따라 나눔

PP1= PP[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC','P_ID', 
         'START_CK','PA', 'AB', 'HOLD', 'INN2', 'BF', 'HIT', 'H2', 'H3',
         'SB', 'HR', 'CS', 'SF', 'BB', 'IB', 'HP', 'KK',
         'GD', 'WP', 'BK', 'ERR', 'R', 'ER', 'P_WHIP_RT', 'P2_WHIP_RT',
         'CB_WHIP_RT','q','K/BB', 'SAP']]

PP2 = PP[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'q','P_ID',
          'START_CK', 'ERA_n', 'ERA_d', 'FIP_n', 'FIP_d', 'KFIP_n', 'KFIP_d', 'WHIP_n',
          'WHIP_d', 'ERC_n', 'ERC_d', 'LOB_n', 'LOB_d', 'KwERA_n',
          'KwERA_d', 'P_BABIP_n', 'P_BABIP_d', 'YANG_n', 'YANG_d']]

PP2['ERA'] = 1
PP2['FIP'] = 1
PP2['KFIP'] = 1
PP2['WHIP'] = 1
PP2['ERC'] = 1
PP2['LOB_RT'] = 1
PP2['KwERA'] = 1
PP2['SAP'] = 1
PP2['P_BABIP'] = 1
PP2['YANG'] = 1


PP1_16_1=PP1[PP1['q']==1]
PP1_16_2=PP1[PP1['q']==2]
PP1_16_3=PP1[PP1['q']==3]
PP1_16_4=PP1[PP1['q']==4]
PP1_16_5=PP1[PP1['q']==5]
PP1_16_6=PP1[PP1['q']==6]
PP1_16_7=PP1[PP1['q']==7]

PP2_16_1=PP2[PP2['q']==1]
PP2_16_2=PP2[PP2['q']==2]
PP2_16_3=PP2[PP2['q']==3]
PP2_16_4=PP2[PP2['q']==4]
PP2_16_5=PP2[PP2['q']==5]
PP2_16_6=PP2[PP2['q']==6]
PP2_16_7=PP2[PP2['q']==7]

PP1_17_1=PP1[PP1['q']==8]
PP1_17_2=PP1[PP1['q']==9]
PP1_17_3=PP1[PP1['q']==10]
PP1_17_4=PP1[PP1['q']==11]
PP1_17_5=PP1[PP1['q']==12]
PP1_17_6=PP1[PP1['q']==13]
PP1_17_7=PP1[PP1['q']==14]

PP2_17_1=PP2[PP2['q']==8]
PP2_17_2=PP2[PP2['q']==9]
PP2_17_3=PP2[PP2['q']==10]
PP2_17_4=PP2[PP2['q']==11]
PP2_17_5=PP2[PP2['q']==12]
PP2_17_6=PP2[PP2['q']==13]
PP2_17_7=PP2[PP2['q']==14]

PP1_18_1=PP1[PP1['q']==15]
PP1_18_2=PP1[PP1['q']==16]
PP1_18_3=PP1[PP1['q']==17]
PP1_18_4=PP1[PP1['q']==18]
PP1_18_5=PP1[PP1['q']==19]
PP1_18_6=PP1[PP1['q']==20]
PP1_18_7=PP1[PP1['q']==21]

PP2_18_1=PP2[PP2['q']==15]
PP2_18_2=PP2[PP2['q']==16]
PP2_18_3=PP2[PP2['q']==17]
PP2_18_4=PP2[PP2['q']==18]
PP2_18_5=PP2[PP2['q']==19]
PP2_18_6=PP2[PP2['q']==20]
PP2_18_7=PP2[PP2['q']==21]

PP1_19_1=PP1[PP1['q']==22]
PP1_19_2=PP1[PP1['q']==23]
PP1_19_3=PP1[PP1['q']==24]
PP1_19_4=PP1[PP1['q']==25]
PP1_19_5=PP1[PP1['q']==26]
PP1_19_6=PP1[PP1['q']==27]
PP1_19_7=PP1[PP1['q']==28]

PP2_19_1=PP2[PP2['q']==22]
PP2_19_2=PP2[PP2['q']==23]
PP2_19_3=PP2[PP2['q']==24]
PP2_19_4=PP2[PP2['q']==25]
PP2_19_5=PP2[PP2['q']==26]
PP2_19_6=PP2[PP2['q']==27]
PP2_19_7=PP2[PP2['q']==28]

PP1_20_1=PP1[PP1['q']==29]
PP1_20_2=PP1[PP1['q']==30]
PP1_20_3=PP1[PP1['q']==31]

PP2_20_1=PP2[PP2['q']==29]
PP2_20_2=PP2[PP2['q']==30]
PP2_20_3=PP2[PP2['q']==31]

In [96]:
# 입력변수 만들 데이터셋 

PP1_16_12=pd.concat([PP1_16_1,PP1_16_2])
PP1_16_23=pd.concat([PP1_16_2,PP1_16_3])
PP1_16_34=pd.concat([PP1_16_3,PP1_16_4])
PP1_16_45=pd.concat([PP1_16_4,PP1_16_5])
PP1_16_56=pd.concat([PP1_16_5,PP1_16_6])

PP2_16_12=pd.concat([PP2_16_1,PP2_16_2])
PP2_16_23=pd.concat([PP2_16_2,PP2_16_3])
PP2_16_34=pd.concat([PP2_16_3,PP2_16_4])
PP2_16_45=pd.concat([PP2_16_4,PP2_16_5])
PP2_16_56=pd.concat([PP2_16_5,PP2_16_6])

PP1_17_12=pd.concat([PP1_17_1,PP1_17_2])
PP1_17_23=pd.concat([PP1_17_2,PP1_17_3])
PP1_17_34=pd.concat([PP1_17_3,PP1_17_4])
PP1_17_45=pd.concat([PP1_17_4,PP1_17_5])
PP1_17_56=pd.concat([PP1_17_5,PP1_17_6])

PP2_17_12=pd.concat([PP2_17_1,PP2_17_2])
PP2_17_23=pd.concat([PP2_17_2,PP2_17_3])
PP2_17_34=pd.concat([PP2_17_3,PP2_17_4])
PP2_17_45=pd.concat([PP2_17_4,PP2_17_5])
PP2_17_56=pd.concat([PP2_17_5,PP2_17_6])

PP1_18_12=pd.concat([PP1_18_1,PP1_18_2])
PP1_18_23=pd.concat([PP1_18_2,PP1_18_3])
PP1_18_34=pd.concat([PP1_18_3,PP1_18_4])
PP1_18_45=pd.concat([PP1_18_4,PP1_18_5])
PP1_18_56=pd.concat([PP1_18_5,PP1_18_6])

PP2_18_12=pd.concat([PP2_18_1,PP2_18_2])
PP2_18_23=pd.concat([PP2_18_2,PP2_18_3])
PP2_18_34=pd.concat([PP2_18_3,PP2_18_4])
PP2_18_45=pd.concat([PP2_18_4,PP2_18_5])
PP2_18_56=pd.concat([PP2_18_5,PP2_18_6])

PP1_19_12=pd.concat([PP1_19_1,PP1_19_2])
PP1_19_23=pd.concat([PP1_19_2,PP1_19_3])
PP1_19_34=pd.concat([PP1_19_3,PP1_19_4])
PP1_19_45=pd.concat([PP1_19_4,PP1_19_5])
PP1_19_56=pd.concat([PP1_19_5,PP1_19_6])

PP2_19_12=pd.concat([PP2_19_1,PP2_19_2])
PP2_19_23=pd.concat([PP2_19_2,PP2_19_3])
PP2_19_34=pd.concat([PP2_19_3,PP2_19_4])
PP2_19_45=pd.concat([PP2_19_4,PP2_19_5])
PP2_19_56=pd.concat([PP2_19_5,PP2_19_6])

PP1_20_12=pd.concat([PP1_20_1,PP1_20_2])
PP1_20_23=pd.concat([PP1_20_2,PP1_20_3])

PP2_20_12=pd.concat([PP2_20_1,PP2_20_2])
PP2_20_23=pd.concat([PP2_20_2,PP2_20_3])

In [97]:
# 분석용 데이터셋 

SP = PP[PP['START_CK']==1]
P1 = SP[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'P_ID','q']]

P1_16_3=P1[P1['q']==3]
P1_16_4=P1[P1['q']==4]
P1_16_5=P1[P1['q']==5]
P1_16_6=P1[P1['q']==6]
P1_16_7=P1[P1['q']==7]

P1_17_3=P1[P1['q']==10]
P1_17_4=P1[P1['q']==11]
P1_17_5=P1[P1['q']==12]
P1_17_6=P1[P1['q']==13]
P1_17_7=P1[P1['q']==14]

P1_18_3=P1[P1['q']==17]
P1_18_4=P1[P1['q']==18]
P1_18_5=P1[P1['q']==19]
P1_18_6=P1[P1['q']==20]
P1_18_7=P1[P1['q']==21]

P1_19_3=P1[P1['q']==24]
P1_19_4=P1[P1['q']==25]
P1_19_5=P1[P1['q']==26]
P1_19_6=P1[P1['q']==27]
P1_19_7=P1[P1['q']==28]

P1_20_3=P1[P1['q']==31]


P2_16_3=P1[P1['q']==3]
P2_16_4=P1[P1['q']==4]
P2_16_5=P1[P1['q']==5]
P2_16_6=P1[P1['q']==6]
P2_16_7=P1[P1['q']==7]

P2_17_3=P1[P1['q']==10]
P2_17_4=P1[P1['q']==11]
P2_17_5=P1[P1['q']==12]
P2_17_6=P1[P1['q']==13]
P2_17_7=P1[P1['q']==14]

P2_18_3=P1[P1['q']==17]
P2_18_4=P1[P1['q']==18]
P2_18_5=P1[P1['q']==19]
P2_18_6=P1[P1['q']==20]
P2_18_7=P1[P1['q']==21]

P2_19_3=P1[P1['q']==24]
P2_19_4=P1[P1['q']==25]
P2_19_5=P1[P1['q']==26]
P2_19_6=P1[P1['q']==27]
P2_19_7=P1[P1['q']==28]

P2_20_3=P1[P1['q']==31]

### 선발투수 평균 입력변수

In [98]:
# 선발투수 평균 입력변수로 변환하는 함수
def PS1_MEAN(dataset):
    
    PPS = dataset[dataset['START_CK']==1]
    PS1_MEAN = PPS.groupby(['P_ID'], as_index=False).mean()
    
    PS1_MEAN = PS1_MEAN.drop(['GDAY_DS','HEADER_NO','START_CK','HOLD','q'], axis=1)
    
    return PS1_MEAN

In [99]:
# 입력변수랑 분석 데이터셋 JOIN

def PS_YEAR_NUM(dataset, dataset2):
    
    dataset = PS1_MEAN(dataset)
    dataset2 = pd.merge(dataset2, dataset, on=['P_ID'], how='left')
    dataset2 = dataset2.drop(['q'],axis=1)
    
    return dataset2

In [100]:
# 분석 데이터셋 중간완성1

PS1_16_3 = PS_YEAR_NUM(PP1_16_12, P1_16_3)
PS1_16_4 = PS_YEAR_NUM(PP1_16_23, P1_16_4)
PS1_16_5 = PS_YEAR_NUM(PP1_16_34, P1_16_5)
PS1_16_6 = PS_YEAR_NUM(PP1_16_45, P1_16_6)
PS1_16_7 = PS_YEAR_NUM(PP1_16_56, P1_16_7)

PS1_17_3 = PS_YEAR_NUM(PP1_17_12, P1_17_3)
PS1_17_4 = PS_YEAR_NUM(PP1_17_23, P1_17_4)
PS1_17_5 = PS_YEAR_NUM(PP1_17_34, P1_17_5)
PS1_17_6 = PS_YEAR_NUM(PP1_17_45, P1_17_6)
PS1_17_7 = PS_YEAR_NUM(PP1_17_56, P1_17_7)

PS1_18_3 = PS_YEAR_NUM(PP1_18_12, P1_18_3)
PS1_18_4 = PS_YEAR_NUM(PP1_18_23, P1_18_4)
PS1_18_5 = PS_YEAR_NUM(PP1_18_34, P1_18_5)
PS1_18_6 = PS_YEAR_NUM(PP1_18_45, P1_18_6)
PS1_18_7 = PS_YEAR_NUM(PP1_18_56, P1_18_7)

PS1_19_3 = PS_YEAR_NUM(PP1_19_12, P1_19_3)
PS1_19_4 = PS_YEAR_NUM(PP1_19_23, P1_19_4)
PS1_19_5 = PS_YEAR_NUM(PP1_19_34, P1_19_5)
PS1_19_6 = PS_YEAR_NUM(PP1_19_45, P1_19_6)
PS1_19_7 = PS_YEAR_NUM(PP1_19_56, P1_19_7)

PS1_20_3 = PS_YEAR_NUM(PP1_20_12, P1_20_3)


In [101]:
# 빌런들 대체하는 선발투수 평균 구하는 함수
def PSV_MEAN(dataset):
    
    PPS=dataset[dataset['START_CK']==1]
    PPS = PPS.drop(['GDAY_DS','HEADER_NO','P_ID','START_CK','HOLD','q'], axis=1)
    PSV_MEAN = PPS.mean()
    
    return PSV_MEAN

In [102]:
# 분석 데이터셋과 JOIN

def PSV_YEAR_NUM(dataset, dataset2):
    
    dataset = PSV_MEAN(dataset)
    dataset2 = dataset2.fillna(dataset)
    
    return dataset2

In [103]:
# 분석 데이터셋 완성1

PS1_16_3 = PSV_YEAR_NUM(PP1_16_12, PS1_16_3)
PS1_16_4 = PSV_YEAR_NUM(PP1_16_23, PS1_16_4)
PS1_16_5 = PSV_YEAR_NUM(PP1_16_34, PS1_16_5)
PS1_16_6 = PSV_YEAR_NUM(PP1_16_45, PS1_16_6)
PS1_16_7 = PSV_YEAR_NUM(PP1_16_56, PS1_16_7)

PS1_17_3 = PSV_YEAR_NUM(PP1_17_12, PS1_17_3)
PS1_17_4 = PSV_YEAR_NUM(PP1_17_23, PS1_17_4)
PS1_17_5 = PSV_YEAR_NUM(PP1_17_34, PS1_17_5)
PS1_17_6 = PSV_YEAR_NUM(PP1_17_45, PS1_17_6)
PS1_17_7 = PSV_YEAR_NUM(PP1_17_56, PS1_17_7)

PS1_18_3 = PSV_YEAR_NUM(PP1_18_12, PS1_18_3)
PS1_18_4 = PSV_YEAR_NUM(PP1_18_23, PS1_18_4)
PS1_18_5 = PSV_YEAR_NUM(PP1_18_34, PS1_18_5)
PS1_18_6 = PSV_YEAR_NUM(PP1_18_45, PS1_18_6)
PS1_18_7 = PSV_YEAR_NUM(PP1_18_56, PS1_18_7)

PS1_19_3 = PSV_YEAR_NUM(PP1_19_12, PS1_19_3)
PS1_19_4 = PSV_YEAR_NUM(PP1_19_23, PS1_19_4)
PS1_19_5 = PSV_YEAR_NUM(PP1_19_34, PS1_19_5)
PS1_19_6 = PSV_YEAR_NUM(PP1_19_45, PS1_19_6)
PS1_19_7 = PSV_YEAR_NUM(PP1_19_56, PS1_19_7)

PS1_20_3 = PSV_YEAR_NUM(PP1_20_12, PS1_20_3)

PS1 = pd.concat([PS1_16_3, PS1_16_4, PS1_16_5, PS1_16_6, PS1_16_7,
                PS1_17_3, PS1_17_4, PS1_17_5, PS1_17_6, PS1_17_7,
                PS1_18_3, PS1_18_4, PS1_18_5, PS1_18_6, PS1_18_7,
                PS1_19_3, PS1_19_4, PS1_19_5, PS1_19_6, PS1_19_7,
                PS1_20_3])
PS1.reset_index(drop=True, inplace=True)

In [104]:
PS1 = PS1.rename({'P_ID': 'PS_P_ID','PA':'PS_PA','AB':'PS_AB','INN2':'PS_INN2','BF':'PS_BF',
                 'HIT':'PS_HIT','H2':'PS_H2','H3':'PS_H3','SB':'PS_SB','HR':'PS_HR',
                 'CS':'PS_CS','SF':'PS_SF','BB':'PS_BB','IB':'PS_IB','HP':'PS_HP','KK':'PS_KK',
                 'GD':'PS_GD','WP':'PS_WP','BK':'PS_BK','ERR':'PS_ERR','R':'PS_R','ER':'PS_ER',
                 'P_WHIP_RT':'PS_P_WHIP_RT','P2_WHIP_RT':'PS_P2_WHIP_RT','CB_WHIP_RT':'PS_CB_WHIP_RT',
                 'K/BB':'PS_K/BB','SAP':'PS_SAP'}, axis='columns')

In [36]:
# 선발투수 평균 입력변수로 변환하는 함수
def PS2_MEAN(dataset):
    
    PPS = dataset[dataset['START_CK']==1]
    PS2_MEAN = PPS.groupby(['P_ID'], as_index=False).sum()
    
    PS2_MEAN = PS2_MEAN.drop(['GDAY_DS','HEADER_NO','START_CK','q'], axis=1)
    
    PS2_MEAN['ERA'] = PS2_MEAN['ERA_n']/PS2_MEAN['ERA_d']
    PS2_MEAN['FIP'] = PS2_MEAN['FIP_n']/PS2_MEAN['FIP_d']
    PS2_MEAN['KFIP'] = PS2_MEAN['KFIP_n']/PS2_MEAN['KFIP_d']
    PS2_MEAN['WHIP'] = PS2_MEAN['WHIP_n']/PS2_MEAN['WHIP_d']
    PS2_MEAN['ERC'] = PS2_MEAN['ERC_n']/PS2_MEAN['ERC_d']
    PS2_MEAN['LOB_RT'] = PS2_MEAN['LOB_n']/PS2_MEAN['LOB_d']
    PS2_MEAN['KwERA'] = PS2_MEAN['KwERA_n']/PS2_MEAN['KwERA_d']
    PS2_MEAN['P_BABIP'] = PS2_MEAN['P_BABIP_n']/PS2_MEAN['P_BABIP_d']
    PS2_MEAN['YANG'] = PS2_MEAN['YANG_n']/PS2_MEAN['YANG_d']
    
    PS2_MEAN = PS2_MEAN.drop(['ERA_n','ERA_d','FIP_n','FIP_d','KFIP_n','KFIP_d',
                           'WHIP_n','WHIP_d','ERC_n','ERC_d','LOB_n','LOB_d',
                            'KwERA_n','KwERA_d','SAP','P_BABIP_n','P_BABIP_d','YANG_n','YANG_d'], axis=1)
    
    return PS2_MEAN

In [37]:
# 입력변수랑 분석 데이터셋 JOIN

def PS2_YEAR_NUM(dataset, dataset2):
    
    dataset = PS2_MEAN(dataset)
    dataset2 = pd.merge(dataset2, dataset, on=['P_ID'], how='left')
    dataset2 = dataset2.drop(['q'],axis=1)
    
    return dataset2

In [115]:
# 분석 데이터셋 중간완성2

PS2_16_3 = PS2_YEAR_NUM(PP2_16_12, P2_16_3)
PS2_16_4 = PS2_YEAR_NUM(PP2_16_23, P2_16_4)
PS2_16_5 = PS2_YEAR_NUM(PP2_16_34, P2_16_5)
PS2_16_6 = PS2_YEAR_NUM(PP2_16_45, P2_16_6)
PS2_16_7 = PS2_YEAR_NUM(PP2_16_56, P2_16_7)

PS2_17_3 = PS2_YEAR_NUM(PP2_17_12, P2_17_3)
PS2_17_4 = PS2_YEAR_NUM(PP2_17_23, P2_17_4)
PS2_17_5 = PS2_YEAR_NUM(PP2_17_34, P2_17_5)
PS2_17_6 = PS2_YEAR_NUM(PP2_17_45, P2_17_6)
PS2_17_7 = PS2_YEAR_NUM(PP2_17_56, P2_17_7)

PS2_18_3 = PS2_YEAR_NUM(PP2_18_12, P2_18_3)
PS2_18_4 = PS2_YEAR_NUM(PP2_18_23, P2_18_4)
PS2_18_5 = PS2_YEAR_NUM(PP2_18_34, P2_18_5)
PS2_18_6 = PS2_YEAR_NUM(PP2_18_45, P2_18_6)
PS2_18_7 = PS2_YEAR_NUM(PP2_18_56, P2_18_7)

PS2_19_3 = PS2_YEAR_NUM(PP2_19_12, P2_19_3)
PS2_19_4 = PS2_YEAR_NUM(PP2_19_23, P2_19_4)
PS2_19_5 = PS2_YEAR_NUM(PP2_19_34, P2_19_5)
PS2_19_6 = PS2_YEAR_NUM(PP2_19_45, P2_19_6)
PS2_19_7 = PS2_YEAR_NUM(PP2_19_56, P2_19_7)

PS2_20_3 = PS2_YEAR_NUM(PP2_20_12, P2_20_3)


In [130]:
# 빌런들 대체하는 선발투수 평균 구하는 함수
def PSV2_MEAN(dataset):
    
    PPS2=dataset[dataset['START_CK']==1]
    PPS2 = PPS2.drop(['G_ID','GDAY_DS','T_ID','VS_T_ID','HEADER_NO','TB_SC','P_ID'], axis=1)
    PSV2_MEAN = PPS2.mean()
    
    return PSV2_MEAN


In [132]:
# 분석 데이터셋과 JOIN

def PSV2_YEAR_NUM(dataset, dataset2):
    
    dataset = PSV2_MEAN(dataset)
    dataset2 = dataset2.fillna(dataset)
    
    return dataset2


In [133]:
# 분석 데이터셋 완성2

PS2_16_3 = PSV2_YEAR_NUM(PP2_16_12, PS2_16_3)
PS2_16_4 = PSV2_YEAR_NUM(PP2_16_23, PS2_16_4)
PS2_16_5 = PSV2_YEAR_NUM(PP2_16_34, PS2_16_5)
PS2_16_6 = PSV2_YEAR_NUM(PP2_16_45, PS2_16_6)
PS2_16_7 = PSV2_YEAR_NUM(PP2_16_56, PS2_16_7)

PS2_17_3 = PSV2_YEAR_NUM(PP2_17_12, PS2_17_3)
PS2_17_4 = PSV2_YEAR_NUM(PP2_17_23, PS2_17_4)
PS2_17_5 = PSV2_YEAR_NUM(PP2_17_34, PS2_17_5)
PS2_17_6 = PSV2_YEAR_NUM(PP2_17_45, PS2_17_6)
PS2_17_7 = PSV2_YEAR_NUM(PP2_17_56, PS2_17_7)

PS2_18_3 = PSV2_YEAR_NUM(PP2_18_12, PS2_18_3)
PS2_18_4 = PSV2_YEAR_NUM(PP2_18_23, PS2_18_4)
PS2_18_5 = PSV2_YEAR_NUM(PP2_18_34, PS2_18_5)
PS2_18_6 = PSV2_YEAR_NUM(PP2_18_45, PS2_18_6)
PS2_18_7 = PSV2_YEAR_NUM(PP2_18_56, PS2_18_7)

PS2_19_3 = PSV2_YEAR_NUM(PP2_19_12, PS2_19_3)
PS2_19_4 = PSV2_YEAR_NUM(PP2_19_23, PS2_19_4)
PS2_19_5 = PSV2_YEAR_NUM(PP2_19_34, PS2_19_5)
PS2_19_6 = PSV2_YEAR_NUM(PP2_19_45, PS2_19_6)
PS2_19_7 = PSV2_YEAR_NUM(PP2_19_56, PS2_19_7)

PS2_20_3 = PSV2_YEAR_NUM(PP2_20_12, PS2_20_3)

PS2 = pd.concat([PS2_16_3, PS2_16_4, PS2_16_5, PS2_16_6, PS2_16_7,
                PS2_17_3, PS2_17_4, PS2_17_5, PS2_17_6, PS2_17_7,
                PS2_18_3, PS2_18_4, PS2_18_5, PS2_18_6, PS2_18_7,
                PS2_19_3, PS2_19_4, PS2_19_5, PS2_19_6, PS2_19_7,
                PS2_20_3])
PS2.reset_index(drop=True, inplace=True)

In [135]:
PS2 = PS2.rename({'P_ID': 'PS_P_ID','ERA':'PS_ERA','FIP':'PS_FIP','KFIP':'PS_KFIP',
                 'WHIP':'PS_WHIP','ERC':'PS_ERC','LOB_RT':'PS_LOB_RT','KwERA':'PS_KwERA',
                 'P_BABIP':'PS_P_BABIP','YANG':'PS_YANG'}, axis='columns')

In [137]:
PS = pd.merge(PS1, PS2, on=['G_ID','GDAY_DS','T_ID','VS_T_ID','HEADER_NO','TB_SC','PS_P_ID'],how='left')

In [139]:
PS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4200 entries, 0 to 4199
Data columns (total 42 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   G_ID           4200 non-null   object 
 1   GDAY_DS        4200 non-null   int64  
 2   T_ID           4200 non-null   object 
 3   VS_T_ID        4200 non-null   object 
 4   HEADER_NO      4200 non-null   int64  
 5   TB_SC          4200 non-null   object 
 6   PS_P_ID        4200 non-null   int64  
 7   PS_PA          4200 non-null   float64
 8   PS_AB          4200 non-null   float64
 9   PS_INN2        4200 non-null   float64
 10  PS_BF          4200 non-null   float64
 11  PS_HIT         4200 non-null   float64
 12  PS_H2          4200 non-null   float64
 13  PS_H3          4200 non-null   float64
 14  PS_SB          4200 non-null   float64
 15  PS_HR          4200 non-null   float64
 16  PS_CS          4200 non-null   float64
 17  PS_SF          4200 non-null   float64
 18  PS_BB   

In [184]:
PS.to_csv('C:/Users/dudco/Desktop/SVT/SVT/PS.csv', sep=',', encoding='ms949')


### 개인타자 파생변수 만들기

#### AVG 타율

In [144]:
# HIT (안타) / AB (타수)

#### OBA 출루율

In [145]:
PH['OBA_n'] = (PH['HIT'] + PH['BB'] + PH['IB'] + PH['HP'])
PH['OBA_d'] = (PH['AB'] + PH['BB'] + PH['IB'] + PH['HP'] + PH['SF'])

#### 경기별 H1(1루타)

In [146]:
PH['H1'] = PH['HIT'] - PH['H2'] - PH['H3'] - PH['HR']

#### SA 장타율

In [147]:
PH['SA_n'] = PH['H1'] + PH['H2']*2 + PH['H3']*3 + PH['HR']*4
PH['SA_d'] = PH['AB']

#### OPS 

In [148]:
# OBA (출루율) + SA (장타율)

#### IsoD 순수 출루율

In [149]:
# OBA (출루율) - AVG (타율)

#### IsoP 순수 장타율(장타력)

In [150]:
# SA (장타율) - AVG (타율)

#### TOT_H 총루타

In [151]:
PH['TOT_H'] = PH['HIT'] - PH['HR']

#### EqA

In [152]:
PH['EqA'] = PH['HIT'] + PH['TOT_H'] + 1.5*(PH['BB'] + PH['HP']) + PH['SB']

#### GPA

In [153]:
# (1.8*OBA + SLG)/4

#### RC

In [154]:
PH['RC'] = PH['HIT'] + PH['BB'] + PH['HP'] - PH['CS'] - PH['GD']

#### BABIP

In [155]:
PH['BABIP_n'] = PH['HIT'] - PH['HR']
PH['BABIP_d'] = PH['AB'] - PH['KK'] - PH['HR'] + PH['SF']

#### SECA 수정타율

In [156]:
PH['SECA_n'] = PH['BB'] + (PH['TOT_H'] - PH['HIT']) + (PH['SB'] - PH['CS'])
PH['SECA_d'] = PH['AB']

#### TA 종합공격력

In [157]:
PH['TA_n'] = PH['TOT_H'] + PH['HP'] + PH['BB'] + PH['SB']
PH['TA_d'] = PH['AB'] - PH['HIT'] + PH['CS'] + PH['GD']

#### PSN 호타준족

In [158]:
PH['PSN_n'] = PH['HR']*PH['SB']*2
PH['PSN_d'] = PH['HR']+PH['SB']

#### XR 추정득점

In [159]:
PH['XR'] = (PH['H1']*0.5) + (PH['H2']*0.72) + (PH['H3']*1.04) + (PH['HR']*1.44) + ((PH['BB']+PH['HP']-PH['IB'])*0.34) + (PH['IB']*0.25) + (PH['SB']*0.18) + (PH['CS']*0.32) - ((PH['PA']-PH['HIT']-PH['KK'])*0.09) - (PH['KK']*0.098) - (PH['GD']*0.37) + (PH['SF']*0.37) + (PH['SH']*0.04)

#### BR 득점공헌

In [160]:
# 주루사를 몰라서 주루사 못 넣음

PH['BR'] = (PH['H1']*0.47) + (PH['H2']*0.78) + (PH['H3']*1.09) + (PH['HR']*1.40) + ((PH['BB'] + PH['HP'])*0.33) + (PH['SB']*0.30) + (PH['CS']*0.60) + ((PH['PA']-PH['HIT'])*0.25)

#### CQ 컨택 퀄리티

In [161]:
PH['CQ_n'] = (PH['HIT']*0.5) + PH['TOT_H']*3
PH['CQ_d'] = PH['PA']-PH['KK']

### 개인타자 7묶음으로 나누기

In [162]:
q=[
    (PH['G_ID']>='20160401HHLG0') & (PH['G_ID']<='20160501NCLT0'),
    (PH['G_ID']>='20160501OBHT0') & (PH['G_ID']<='20160526SKNC0'),
    (PH['G_ID']>='20160527LGOB0') & (PH['G_ID']<='20160619SKLT0'),
    (PH['G_ID']>='20160619WOHH0') & (PH['G_ID']<='20160720SSOB0'),
    (PH['G_ID']>='20160721HTLT0') & (PH['G_ID']<='20160814HHHT0'),
    (PH['G_ID']>='20160814KTNC0') & (PH['G_ID']<='20160908WOSK0'),
    (PH['G_ID']>='20160909KTHH0') & (PH['G_ID']<='20161009WOLT0'),
    (PH['G_ID']>='20170331HHOB0') & (PH['G_ID']<='20170428WOHH0'),
    (PH['G_ID']>='20170429LGKT0') & (PH['G_ID']<='20170524HTHH0'),
    (PH['G_ID']>='20170524KTSS0') & (PH['G_ID']<='20170616LTWO0'),
    (PH['G_ID']>='20170616NCOB0') & (PH['G_ID']<='20170712WOOB0'),
    (PH['G_ID']>='20170713LGSK0') & (PH['G_ID']<='20170809NCSK0'),
    (PH['G_ID']>='20170809WOHT0') & (PH['G_ID']<='20170903LGNC0'),
    (PH['G_ID']>='20170903SKKT0') & (PH['G_ID']<='20171003WOSS0'),
    (PH['G_ID']>='20180324HHWO0') & (PH['G_ID']<='20180422LGNC0'),
    (PH['G_ID']>='20180422SKLT0') & (PH['G_ID']<='20180518SSWO0'),
    (PH['G_ID']>='20180519HHLG0') & (PH['G_ID']<='20180612KTOB0'),
    (PH['G_ID']>='20180612LGNC0') & (PH['G_ID']<='20180707NCWO0'),
    (PH['G_ID']>='20180707SSOB0') & (PH['G_ID']<='20180804SSLT0'),
    (PH['G_ID']>='20180804WOKT0') & (PH['G_ID']<='20180915SKHT0'),
    (PH['G_ID']>='20180915SSKT0') & (PH['G_ID']<='20181014OBLT0'),
    (PH['G_ID']>='20190323HHOB0') & (PH['G_ID']<='20190420WOLG0'),
    (PH['G_ID']>='20190421KTLT0') & (PH['G_ID']<='20190516LGLT0'),
    (PH['G_ID']>='20190516SKNC0') & (PH['G_ID']<='20190608WOOB0'),
    (PH['G_ID']>='20190609HTNC0') & (PH['G_ID']<='20190704LTSK0'),
    (PH['G_ID']>='20190704NCHT0') & (PH['G_ID']<='20190807LGHT0'),
    (PH['G_ID']>='20190807SSNC0') & (PH['G_ID']<='20190831SSOB0'),
    (PH['G_ID']>='20190831WONC0') & (PH['G_ID']<='20191001WOLT0'),
    (PH['G_ID']>='20200505HHSK0') & (PH['G_ID']<='20200602SKNC0'),
    (PH['G_ID']>='20200602SSLG0') & (PH['G_ID']<='20200626HTWO0'),
    (PH['G_ID']>='20200626KTHH0') & (PH['G_ID']<='20200719WOSK0')

]
choicelist =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
PH['q']=np.select(q, choicelist)

In [163]:
# 분모, 분자 상관유무에 따라 나눔

PH1 = PH[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'P_ID','START_CK','q',
            'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'H1', 'SH', 'SF',
            'BB','IB','HP', 'KK', 'GD','ERR', 'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN','TOT_H',
            'XR', 'BR','RC']]

PH2 = PH[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'P_ID','START_CK','q',
             'HIT','AB','PA', 'RBI', 'RUN', 'H2', 'H3', 'HR', 'SB', 'CS', 'H1', 'SH', 'SF',
             'BB','IB','HP', 'KK', 'GD', 'OBA_n', 'OBA_d', 'SA_n', 'SA_d','BABIP_n', 'BABIP_d', 'SECA_n','SECA_d', 
             'TA_n', 'TA_d', 'PSN_n', 'PSN_d', 'CQ_n', 'CQ_d','TOT_H']]

PH2['AVG'] = 1
PH2['OBA'] = 1
PH2['SA'] = 1
PH2['BABIP'] = 1
PH2['SECA'] = 1
PH2['TA'] = 1
PH2['PSN'] = 1
PH2['CQ'] = 1
PH2['OPS'] = 1
PH2['IsoD'] = 1
PH2['IsoP'] = 1
PH2['GPA'] = 1


PH1_16_1=PH1[PH1['q']==1]
PH1_16_2=PH1[PH1['q']==2]
PH1_16_3=PH1[PH1['q']==3]
PH1_16_4=PH1[PH1['q']==4]
PH1_16_5=PH1[PH1['q']==5]
PH1_16_6=PH1[PH1['q']==6]
PH1_16_7=PH1[PH1['q']==7]

PH2_16_1=PH2[PH2['q']==1]
PH2_16_2=PH2[PH2['q']==2]
PH2_16_3=PH2[PH2['q']==3]
PH2_16_4=PH2[PH2['q']==4]
PH2_16_5=PH2[PH2['q']==5]
PH2_16_6=PH2[PH2['q']==6]
PH2_16_7=PH2[PH2['q']==7]

PH1_17_1=PH1[PH1['q']==8]
PH1_17_2=PH1[PH1['q']==9]
PH1_17_3=PH1[PH1['q']==10]
PH1_17_4=PH1[PH1['q']==11]
PH1_17_5=PH1[PH1['q']==12]
PH1_17_6=PH1[PH1['q']==13]
PH1_17_7=PH1[PH1['q']==14]

PH2_17_1=PH2[PH2['q']==8]
PH2_17_2=PH2[PH2['q']==9]
PH2_17_3=PH2[PH2['q']==10]
PH2_17_4=PH2[PH2['q']==11]
PH2_17_5=PH2[PH2['q']==12]
PH2_17_6=PH2[PH2['q']==13]
PH2_17_7=PH2[PH2['q']==14]

PH1_18_1=PH1[PH1['q']==15]
PH1_18_2=PH1[PH1['q']==16]
PH1_18_3=PH1[PH1['q']==17]
PH1_18_4=PH1[PH1['q']==18]
PH1_18_5=PH1[PH1['q']==19]
PH1_18_6=PH1[PH1['q']==20]
PH1_18_7=PH1[PH1['q']==21]

PH2_18_1=PH2[PH2['q']==15]
PH2_18_2=PH2[PH2['q']==16]
PH2_18_3=PH2[PH2['q']==17]
PH2_18_4=PH2[PH2['q']==18]
PH2_18_5=PH2[PH2['q']==19]
PH2_18_6=PH2[PH2['q']==20]
PH2_18_7=PH2[PH2['q']==21]

PH1_19_1=PH1[PH1['q']==22]
PH1_19_2=PH1[PH1['q']==23]
PH1_19_3=PH1[PH1['q']==24]
PH1_19_4=PH1[PH1['q']==25]
PH1_19_5=PH1[PH1['q']==26]
PH1_19_6=PH1[PH1['q']==27]
PH1_19_7=PH1[PH1['q']==28]

PH2_19_1=PH2[PH2['q']==22]
PH2_19_2=PH2[PH2['q']==23]
PH2_19_3=PH2[PH2['q']==24]
PH2_19_4=PH2[PH2['q']==25]
PH2_19_5=PH2[PH2['q']==26]
PH2_19_6=PH2[PH2['q']==27]
PH2_19_7=PH2[PH2['q']==28]

PH1_20_1=PH1[PH1['q']==29]
PH1_20_2=PH1[PH1['q']==30]
PH1_20_3=PH1[PH1['q']==31]

PH2_20_1=PH2[PH2['q']==29]
PH2_20_2=PH2[PH2['q']==30]
PH2_20_3=PH2[PH2['q']==31]

In [164]:
# 입력변수 만들 데이터셋 

PH1_16_12=pd.concat([PH1_16_1,PH1_16_2])
PH1_16_23=pd.concat([PH1_16_2,PH1_16_3])
PH1_16_34=pd.concat([PH1_16_3,PH1_16_4])
PH1_16_45=pd.concat([PH1_16_4,PH1_16_5])
PH1_16_56=pd.concat([PH1_16_5,PH1_16_6])

PH2_16_12=pd.concat([PH2_16_1,PH2_16_2])
PH2_16_23=pd.concat([PH2_16_2,PH2_16_3])
PH2_16_34=pd.concat([PH2_16_3,PH2_16_4])
PH2_16_45=pd.concat([PH2_16_4,PH2_16_5])
PH2_16_56=pd.concat([PH2_16_5,PH2_16_6])

PH1_17_12=pd.concat([PH1_17_1,PH1_17_2])
PH1_17_23=pd.concat([PH1_17_2,PH1_17_3])
PH1_17_34=pd.concat([PH1_17_3,PH1_17_4])
PH1_17_45=pd.concat([PH1_17_4,PH1_17_5])
PH1_17_56=pd.concat([PH1_17_5,PH1_17_6])

PH2_17_12=pd.concat([PH2_17_1,PH2_17_2])
PH2_17_23=pd.concat([PH2_17_2,PH2_17_3])
PH2_17_34=pd.concat([PH2_17_3,PH2_17_4])
PH2_17_45=pd.concat([PH2_17_4,PH2_17_5])
PH2_17_56=pd.concat([PH2_17_5,PH2_17_6])

PH1_18_12=pd.concat([PH1_18_1,PH1_18_2])
PH1_18_23=pd.concat([PH1_18_2,PH1_18_3])
PH1_18_34=pd.concat([PH1_18_3,PH1_18_4])
PH1_18_45=pd.concat([PH1_18_4,PH1_18_5])
PH1_18_56=pd.concat([PH1_18_5,PH1_18_6])

PH2_18_12=pd.concat([PH2_18_1,PH2_18_2])
PH2_18_23=pd.concat([PH2_18_2,PH2_18_3])
PH2_18_34=pd.concat([PH2_18_3,PH2_18_4])
PH2_18_45=pd.concat([PH2_18_4,PH2_18_5])
PH2_18_56=pd.concat([PH2_18_5,PH2_18_6])

PH1_19_12=pd.concat([PH1_19_1,PH1_19_2])
PH1_19_23=pd.concat([PH1_19_2,PH1_19_3])
PH1_19_34=pd.concat([PH1_19_3,PH1_19_4])
PH1_19_45=pd.concat([PH1_19_4,PH1_19_5])
PH1_19_56=pd.concat([PH1_19_5,PH1_19_6])

PH2_19_12=pd.concat([PH2_19_1,PH2_19_2])
PH2_19_23=pd.concat([PH2_19_2,PH2_19_3])
PH2_19_34=pd.concat([PH2_19_3,PH2_19_4])
PH2_19_45=pd.concat([PH2_19_4,PH2_19_5])
PH2_19_56=pd.concat([PH2_19_5,PH2_19_6])

PH1_20_12=pd.concat([PH1_20_1,PH1_20_2])
PH1_20_23=pd.concat([PH1_20_2,PH1_20_3])

PH2_20_12=pd.concat([PH2_20_1,PH2_20_2])
PH2_20_23=pd.concat([PH2_20_2,PH2_20_3])

In [165]:
# 분석용 데이터셋 

SH = PH[PH['START_CK']==1]
H1 = SH[['G_ID', 'T_ID','q']]
H1 = H1.drop_duplicates(['G_ID','T_ID'], keep='first')

H1_16_3=H1[H1['q']==3]
H1_16_4=H1[H1['q']==4]
H1_16_5=H1[H1['q']==5]
H1_16_6=H1[H1['q']==6]
H1_16_7=H1[H1['q']==7]

H1_17_3=H1[H1['q']==10]
H1_17_4=H1[H1['q']==11]
H1_17_5=H1[H1['q']==12]
H1_17_6=H1[H1['q']==13]
H1_17_7=H1[H1['q']==14]

H1_18_3=H1[H1['q']==17]
H1_18_4=H1[H1['q']==18]
H1_18_5=H1[H1['q']==19]
H1_18_6=H1[H1['q']==20]
H1_18_7=H1[H1['q']==21]

H1_19_3=H1[H1['q']==24]
H1_19_4=H1[H1['q']==25]
H1_19_5=H1[H1['q']==26]
H1_19_6=H1[H1['q']==27]
H1_19_7=H1[H1['q']==28]

H1_20_3=H1[H1['q']==31]


H2_16_3=H1[H1['q']==3]
H2_16_4=H1[H1['q']==4]
H2_16_5=H1[H1['q']==5]
H2_16_6=H1[H1['q']==6]
H2_16_7=H1[H1['q']==7]

H2_17_3=H1[H1['q']==10]
H2_17_4=H1[H1['q']==11]
H2_17_5=H1[H1['q']==12]
H2_17_6=H1[H1['q']==13]
H2_17_7=H1[H1['q']==14]

H2_18_3=H1[H1['q']==17]
H2_18_4=H1[H1['q']==18]
H2_18_5=H1[H1['q']==19]
H2_18_6=H1[H1['q']==20]
H2_18_7=H1[H1['q']==21]

H2_19_3=H1[H1['q']==24]
H2_19_4=H1[H1['q']==25]
H2_19_5=H1[H1['q']==26]
H2_19_6=H1[H1['q']==27]
H2_19_7=H1[H1['q']==28]

H2_20_3=H1[H1['q']==31]

### 개인타자_ 상대선발투수 고려한 평균

In [166]:
# 개인타자에 가중할 선발투수 ERA 
def PH_PS(dataset):
    
    
    P = dataset[['G_ID','GDAY_DS','T_ID','VS_T_ID','P_ID','ERA']]
    
    P['MY_T_ID'] = P['T_ID']
    P['T_ID'] = P['VS_T_ID']
    P['VS_T_ID'] = P['MY_T_ID']
    P = P.rename({'P_ID':'SP_ID'},axis=1)
    PH_PS = P.drop(['MY_T_ID'],axis=1)
    
    return PH_PS

In [167]:
# 팀별 선발타자 평균 입력변수로 변환하는 함수1
def HS1_MEAN(dataset):
    
    PHS = dataset[dataset['START_CK']==1]
    HS1_MEAN = PHS.groupby(['G_ID', 'T_ID'], as_index=False).sum()
    HS1_MEAN = HS1_MEAN.groupby(['T_ID'], as_index=False).mean()
    
    HS1_MEAN = HS1_MEAN.drop(['GDAY_DS','HEADER_NO','P_ID','START_CK','q'], axis=1)
    
    return HS1_MEAN

In [168]:
# 입력변수랑 분석 데이터셋 JOIN

def HS1_YEAR_NUM(dataset, dataset2, dataset3):
    
    dataset = HS1_MEAN(dataset)
    dataset2 = pd.merge(dataset2, dataset, on=['T_ID'], how='left')
    dataset2 = dataset2.drop(['q'],axis=1)
    dataset3 = PH_PS(dataset3)
    dataset3 = pd.merge(dataset2, dataset3, on=['G_ID','T_ID'], how='left')
    
    return dataset3

In [169]:
# 분석 데이터셋 완성1

HS1_16_3 = HS1_YEAR_NUM(PH1_16_12, H1_16_3, PS2_16_3)
HS1_16_4 = HS1_YEAR_NUM(PH1_16_23, H1_16_4, PS2_16_4)
HS1_16_5 = HS1_YEAR_NUM(PH1_16_34, H1_16_5, PS2_16_5)
HS1_16_6 = HS1_YEAR_NUM(PH1_16_45, H1_16_6, PS2_16_6)
HS1_16_7 = HS1_YEAR_NUM(PH1_16_56, H1_16_7, PS2_16_7)

HS1_17_3 = HS1_YEAR_NUM(PH1_17_12, H1_17_3, PS2_17_3)
HS1_17_4 = HS1_YEAR_NUM(PH1_17_23, H1_17_4, PS2_17_4)
HS1_17_5 = HS1_YEAR_NUM(PH1_17_34, H1_17_5, PS2_17_5)
HS1_17_6 = HS1_YEAR_NUM(PH1_17_45, H1_17_6, PS2_17_6)
HS1_17_7 = HS1_YEAR_NUM(PH1_17_56, H1_17_7, PS2_17_7)

HS1_18_3 = HS1_YEAR_NUM(PH1_18_12, H1_18_3, PS2_18_3)
HS1_18_4 = HS1_YEAR_NUM(PH1_18_23, H1_18_4, PS2_18_4)
HS1_18_5 = HS1_YEAR_NUM(PH1_18_34, H1_18_5, PS2_18_5)
HS1_18_6 = HS1_YEAR_NUM(PH1_18_45, H1_18_6, PS2_18_6)
HS1_18_7 = HS1_YEAR_NUM(PH1_18_56, H1_18_7, PS2_18_7)

HS1_19_3 = HS1_YEAR_NUM(PH1_19_12, H1_19_3, PS2_19_3)
HS1_19_4 = HS1_YEAR_NUM(PH1_19_23, H1_19_4, PS2_19_4)
HS1_19_5 = HS1_YEAR_NUM(PH1_19_34, H1_19_5, PS2_19_5)
HS1_19_6 = HS1_YEAR_NUM(PH1_19_45, H1_19_6, PS2_19_6)
HS1_19_7 = HS1_YEAR_NUM(PH1_19_56, H1_19_7, PS2_19_7)

HS1_20_3 = HS1_YEAR_NUM(PH1_20_12, H1_20_3, PS2_20_3)

HS1 = pd.concat([HS1_16_3, HS1_16_4, HS1_16_5, HS1_16_6, HS1_16_7,
                HS1_17_3, HS1_17_4, HS1_17_5, HS1_17_6, HS1_17_7,
                HS1_18_3, HS1_18_4, HS1_18_5, HS1_18_6, HS1_18_7,
                HS1_19_3, HS1_19_4, HS1_19_5, HS1_19_6, HS1_19_7,
                HS1_20_3])
HS1.reset_index(drop=True, inplace=True)

In [172]:
HS1 = HS1.rename({'PA':'HS_PA','AB':'HS_AB','RBI':'HS_RBI','RUN':'HS_RUN',
                 'HIT':'HS_HIT','H2':'HS_H2','H3':'HS_H3','HR':'HS_HR','SB':'HS_SB',
                 'CS':'HS_CS','H1':'HS_H1','SH':'HS_SH','SF':'HS_SF','BB':'HS_BB','IB':'HS_IB',
                  'HP':'HS_HP','KK':'HS_KK','GD':'HS_GD','ERR':'HS_ERR','LOB':'HS_LOB',
                  'P_HRA_RT':'HS_P_HRA_RT','P_AB_CN':'HS_P_AB_CN','P_HIT_CN':'HS_P_HIT_CN',
                  'TOT_H':'HS_TOT_H','XR':'HS_XR','BR':'HS_BR','RC':'HS_RC','ERA':'SP_ERA'}, axis='columns')

In [173]:
# 팀별 선발타자 평균 입력변수로 변환하는 함수2
def HS2_MEAN(dataset):
    
    PHS = dataset[dataset['START_CK']==1]
    HS2_MEAN = PHS.groupby(['G_ID', 'T_ID'], as_index=False).sum()
    HS2_MEAN = HS2_MEAN.groupby(['T_ID'], as_index=False).mean()
   
    HS2_MEAN['AVG'] = HS2_MEAN['HIT']/HS2_MEAN['AB']
    HS2_MEAN['OBA'] = HS2_MEAN['OBA_n']/HS2_MEAN['OBA_d']
    HS2_MEAN['SA'] = HS2_MEAN['SA_n']/HS2_MEAN['SA_d']
    HS2_MEAN['OPS'] = HS2_MEAN['OBA']+HS2_MEAN['SA']
    HS2_MEAN['IsoD'] = (HS2_MEAN['OBA'])-(HS2_MEAN['AVG'])
    HS2_MEAN['IsoP'] = (HS2_MEAN['SA'])-(HS2_MEAN['AVG'])
    HS2_MEAN['GPA'] = (1.8*HS2_MEAN['OBA']+HS2_MEAN['SA'])/4
    HS2_MEAN['BABIP'] = HS2_MEAN['BABIP_n']/HS2_MEAN['BABIP_d']
    HS2_MEAN['SECA'] = HS2_MEAN['SECA_n']/HS2_MEAN['SECA_d']
    HS2_MEAN['TA'] = HS2_MEAN['TA_n']/HS2_MEAN['TA_d']
    HS2_MEAN['PSN'] = HS2_MEAN['PSN_n']/HS2_MEAN['PSN_d']
    HS2_MEAN['CQ'] = HS2_MEAN['CQ_n']/HS2_MEAN['CQ_d']
    
    HS2_MEAN = HS2_MEAN.drop(['GDAY_DS','HEADER_NO','P_ID','START_CK','q',
                              'PA','AB','RBI','RUN','HIT','H2','H3','HR','SB',
                              'CS','H1','SH','SF','BB','IB','HP','KK','GD','TOT_H',
                              'OBA_n','OBA_d','SA_n','SA_d','BABIP_n','BABIP_d','SECA_n',
                              'SECA_d','TA_n','TA_d','PSN_n','PSN_d','CQ_n','CQ_d'], axis=1)
    
    return HS2_MEAN

In [174]:
# 입력변수랑 분석 데이터셋 JOIN

def HS2_YEAR_NUM(dataset, dataset2, dataset3):
    
    dataset = HS2_MEAN(dataset)
    dataset2 = pd.merge(dataset2, dataset, on=['T_ID'], how='left')
    dataset2 = dataset2.drop(['q'],axis=1)
    dataset3 = PH_PS(dataset3)
    dataset3 = pd.merge(dataset2, dataset3, on=['G_ID','T_ID'], how='left')
        
    return dataset3

In [175]:
# 분석 데이터셋 완성2

HS2_16_3 = HS2_YEAR_NUM(PH2_16_12, H2_16_3, PS2_16_3)
HS2_16_4 = HS2_YEAR_NUM(PH2_16_23, H2_16_4, PS2_16_4)
HS2_16_5 = HS2_YEAR_NUM(PH2_16_34, H2_16_5, PS2_16_5)
HS2_16_6 = HS2_YEAR_NUM(PH2_16_45, H2_16_6, PS2_16_6)
HS2_16_7 = HS2_YEAR_NUM(PH2_16_56, H2_16_7, PS2_16_7)

HS2_17_3 = HS2_YEAR_NUM(PH2_17_12, H2_17_3, PS2_17_3)
HS2_17_4 = HS2_YEAR_NUM(PH2_17_23, H2_17_4, PS2_17_4)
HS2_17_5 = HS2_YEAR_NUM(PH2_17_34, H2_17_5, PS2_17_5)
HS2_17_6 = HS2_YEAR_NUM(PH2_17_45, H2_17_6, PS2_17_6)
HS2_17_7 = HS2_YEAR_NUM(PH2_17_56, H2_17_7, PS2_17_7)

HS2_18_3 = HS2_YEAR_NUM(PH2_18_12, H2_18_3, PS2_18_3)
HS2_18_4 = HS2_YEAR_NUM(PH2_18_23, H2_18_4, PS2_18_4)
HS2_18_5 = HS2_YEAR_NUM(PH2_18_34, H2_18_5, PS2_18_5)
HS2_18_6 = HS2_YEAR_NUM(PH2_18_45, H2_18_6, PS2_18_6)
HS2_18_7 = HS2_YEAR_NUM(PH2_18_56, H2_18_7, PS2_18_7)

HS2_19_3 = HS2_YEAR_NUM(PH2_19_12, H2_19_3, PS2_19_3)
HS2_19_4 = HS2_YEAR_NUM(PH2_19_23, H2_19_4, PS2_19_4)
HS2_19_5 = HS2_YEAR_NUM(PH2_19_34, H2_19_5, PS2_19_5)
HS2_19_6 = HS2_YEAR_NUM(PH2_19_45, H2_19_6, PS2_19_6)
HS2_19_7 = HS2_YEAR_NUM(PH2_19_56, H2_19_7, PS2_19_7)

HS2_20_3 = HS2_YEAR_NUM(PH2_20_12, H2_20_3, PS2_20_3)

HS2 = pd.concat([HS2_16_3, HS2_16_4, HS2_16_5, HS2_16_6, HS2_16_7,
                HS2_17_3, HS2_17_4, HS2_17_5, HS2_17_6, HS2_17_7,
                HS2_18_3, HS2_18_4, HS2_18_5, HS2_18_6, HS2_18_7,
                HS2_19_3, HS2_19_4, HS2_19_5, HS2_19_6, HS2_19_7,
                HS2_20_3])
HS2.reset_index(drop=True, inplace=True)

In [178]:
HS2 = HS2.rename({'AVG':'HS_AVG','OBA':'HS_OBA','SA':'HS_SA','BABIP':'HS_BABIP',
                 'SECA':'HS_SECA','TA':'HS_TA','PSN':'HS_PSN','CQ':'HS_CQ',
                 'OPS':'HS_OPS','IsoD':'HS_IsoD','IsoP':'HS_IsoP','GPA':'HS_GPA','ERA':'SP_ERA'}, axis='columns')

In [179]:
HS = pd.merge(HS1, HS2, on=['G_ID','GDAY_DS','T_ID','VS_T_ID','SP_ID','SP_ERA'],how='left')

In [183]:
HS = HS.drop(['GDAY_DS','VS_T_ID','SP_ID'],axis=1)

In [185]:
HS.to_csv('C:/Users/dudco/Desktop/SVT/SVT/HS.csv', sep=',', encoding='ms949')
